# mRNA Reconstruction using RNA-Seq Next-Generation Sequencing Data
The goal of this project is to reconstruct mRNA from RNA-Seq data. <br>
In this specific situation, the RNA-Seq was extracted from healthy patients and patients diagnosed with Type 2 Diabetes. <br>
Thus, this project examines the preproinsulin mRNA (pre-precursor of the insulin hormone/protein) of pancreatic beta cells.<br>

### How to Run:
Check all the imports in the next cell and install all packages you are missing.<br>
Then simply run the cells.<br>
Smaller necessary files are included in this GitHub repository while other files will be downloaded from this notebook.<br>
Download speeds from the European Bioinformatics Institute vary and could take a while.<br>
Estimated time to completion: About a week<br>
The final products of most interest are probably in these folders: Counts, Reconstruct, and Summary.<br>

#### System Requirements:
This project requires about 40 GBs of storage (NGS data are large files).<br>
RAM storage will likely not be a large issue for most computers.<br><br>



For more general information about this project including biological interpretations, see my Medium article:<br>
https://daovang.medium.com/simple-reconstruction-of-mrna-from-next-generation-sequencing-rna-seq-c4faaa5da90d?source=friends_link&sk=ee61abdaa22a0773f5030d08d47de577<br>

All reference preproinsulin data was obtained from NCBI:<br>
https://www.ncbi.nlm.nih.gov/gene?term=INS%5BGene%5D%20AND%20%22Homo%20sapiens%22%5BOrganism%5D&cmd=DetailsSearch<br>

All NGS data is obtained from the European Bioinformatics Institute:<br>
https://medium.com/r/?url=https%3A%2F%2Fwww.ebi.ac.uk%2Farrayexpress%2Fexperiments%2FE-MTAB-5061%2F<br>

The original purpose of the data can be found in this publication in Cell:<br>
https://www.cell.com/cell-metabolism/fulltext/S1550-4131(16)30436-3<br><br>

Version 1.0<br>
11/5/2020<br>

In [ ]:
import Bio # This is the biopython package
from Bio import SeqIO, pairwise2
from Bio.Seq import Seq

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from urllib.request import urlretrieve, urlopen

import urllib, gzip, shutil, os

In [ ]:
# Loading Reference Sequences:
# All reference sequences are obtained from NCBI

## Insulin Variant 1
var_1_ref_mRNA_cDNA = SeqIO.read("Insulin Reference mRNA and Protein Sequences\\NM_000207.3.fa", "fasta")

## Insulin Variant 2
var_2_ref_mRNA_cDNA = SeqIO.read("Insulin Reference mRNA and Protein Sequences\\NM_001185097.2.fa", "fasta")

## Insulin Variant 3
var_3_ref_mRNA_cDNA = SeqIO.read("Insulin Reference mRNA and Protein Sequences\\NM_001185098.2.fa", "fasta")

## Insulin Variant 4
var_4_ref_mRNA_cDNA = SeqIO.read("Insulin Reference mRNA and Protein Sequences\\NM_001291897.2.fa", "fasta")


## Preproinsulin Reference Sequence:
#### Note: All variants mRNA create the same protein/protein sequence (preproinsulin)
ref_protein = SeqIO.read("Insulin Reference mRNA and Protein Sequences\\NP_000198.1.fa", "fasta")


In [ ]:
''' 
Downloading RNA-Seq/transcriptomics data from https://www.ebi.ac.uk/arrayexpress/experiments/E-MTAB-5061/samples/?full=true:

See for experiment details: https://data.humancellatlas.org/explore/projects/ae71be1d-ddd8-4feb-9bed-24c3ddb6e1ad 
    This experiment ("Single-cell RNA-seq analysis of human pancreas from healthy individuals and type 2 diabetes patients")
    only examines the RNA found in single pancreatic cells using RNA-Seq.
        Although the experiment does collect RNA from all pancreatic cell types, I am only interested in beta cells because
        these cells are the only insulin producing cells.

Samples Collected from 10 Individuals:
    6 Healthy Individuals
    4 Individuals with Type 2 Diabetes (T2D)
Download Size = 9.44 GBs (compressed files)
270 FASTQ Files = 36.9 GBs (extracted/decompressed files)
'''

# Downloading the metadata/summary of all the samples collected in the experiment:
if os.path.exists('RNA-Seq') != True: ## Creating a RNA-Seq folder
    os.mkdir('RNA-Seq')
    
urlretrieve('https://www.ebi.ac.uk/arrayexpress/files/E-MTAB-5061/E-MTAB-5061.sdrf.txt', 'RNA-Seq\\metadata.txt')    

sample_reference = pd.read_csv('RNA-Seq\\metadata.txt', sep='\t')

## Splicing only specific rows from the metadata:
### The first filter is to make sure the quality is "OK":
sample_reference = sample_reference[sample_reference['Characteristics[single cell well quality]'] == 'OK']

### The second filter is to select only for beta cells (insulin producing cells):
sample_reference = sample_reference[sample_reference['Characteristics[cell type]'] == 'beta cell']



# Downloading FASTQ files using the spliced metadata table:
## Note: This may take long since download speeds with servers that hold biological information are typically slow

counter = 0  # Setting up a counter for the tracking
for index, row in sample_reference.iterrows():
    print(row['Source Name'])
    
    ### The following below is a copy-and-paste from: https://stackoverflow.com/questions/22676/how-do-i-download-a-file-over-http-using-python
    #### This is from the solution that adds in a "progress bar", which is important in this case because of slow download
    #### speeds--ensuring that the connection to the server is still running.
    url = row['Comment[FASTQ_URI]']

    file_name = 'RNA-Seq\\' + row['Source Name'] + '.fastq.gz'
    u = urlopen(url)
    f = open(file_name, 'wb')
    meta = u.info()
    file_size = int(dict(meta.items())['Content-length'])
    print ("Downloading: %s Bytes: %s" % (file_name, file_size))

    file_size_dl = 0
    block_sz = 8192
    while True:
        buffer = u.read(block_sz)
        if not buffer:
            break

        file_size_dl += len(buffer)
        f.write(buffer)
        status = r"%10d  [%3.2f%%]" % (file_size_dl, file_size_dl * 100. / file_size)
        status = status + chr(8)*(len(status)+1)
        print(status)

    f.close()
    
    ## This extracting the compressed file:
    ### The following below is a copy-and-paste from: https://stackoverflow.com/questions/31028815/how-to-unzip-gz-file-using-python
    with gzip.open('RNA-Seq\\' + row['Source Name'] + '.fastq.gz', 'rb') as f_in:
        with open('RNA-Seq\\' + row['Source Name'] + '.fastq', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    
    ## Removing the compressed .gz file:
    os.remove('RNA-Seq\\' + row['Source Name'] + '.fastq.gz') 
    
    counter += 1
    print(str(counter / len(sample_reference) * 100) + "% Samples Downloaded" )

In [ ]:
'''
Finding the RNA sequences that match the mRNA of Variant 3 for reconstructing the mRNA preproinsulin sequence for each
individual.

Because the RNA-Seq reads are all in a length of 43 nucleotides, there are 3 possiblities where the reads can be
matched in the preproinsulin mRNA reference sequence (644 nucleotides):

    Also, indels (frameshifts) are not detected in this because the two final insulin chains (A and B) would most 
    likely NOT be functional as a hormone--the insulin receptor seems to bind to the entire insuline protein.
        https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3793637/

''' 

## Splitting the reference sequence of preproinsulin variant 3 by 20 nucleotides and creating a DataFrame for matches:
variant_3 = str(var_3_ref_mRNA_cDNA.seq)
var_3_length = len(var_3_ref_mRNA_cDNA.seq)  ## Number of nucleotides in variant 3

## Grabbing all the FASTQ files downloaded:
fastq_list = os.listdir('RNA-Seq\\')
fastq_list.remove('metadata.txt')

match_requirement = 35  ### Setting the matching requirement to be 35 nucleotides

reconstruct_check_list = list(range(9, 42))
reconstruct_check_list.extend(list(range(224, 634))) ### Checking for variant 1 exons, but skipping the first and last few


if os.path.exists('Alignments') != True: ## Creating a RNA-Seq folder
    os.mkdir('Alignments')

for file in fastq_list:
    fastq = SeqIO.parse('RNA-Seq\\' + file, "fastq")
    print(file)
    
    reconstruct_df = pd.DataFrame(columns= list(range(0, 644))) ## Creating a DataFrame to store the matches
    
    index_counter = 0
    reconstruct_counter = 0
    for seq_record in fastq:
        seq = str(seq_record.seq)
        
        matches = pairwise2.align.localms(variant_3, seq, 1, -1, -99, -99)  ### Matching & Aligning
        
        if matches[0].score >= match_requirement:  ### Aligning and saving to DataFrame
            pair = matches[0]  ### Only taking the first alignment
            
            if "-" not in pair.seqA: ### 
                match_seq = list(pair.seqB)
                start_index = pair.start
                end_index = pair.end

                match_index = start_index            
                for z in range(start_index, end_index):
                    align_seq = match_seq[match_index]
                    
                    if align_seq == '-':  ### Gaps are replaced with np.NaN
                        align_seq = np.NaN
                        
                    reconstruct_df.loc[index_counter, match_index] = align_seq
                    match_index += 1
                
                index_counter += 1
                reconstruct_counter += 1
                                
        if reconstruct_counter == 1000:  ### Every 1000 matches, checking if each aligned nucleotide has at least 25 matches
            finish_check = 0
            print('check')
            for u in reconstruct_check_list:
                if len(reconstruct_df[u].dropna()) >= 25:  
                    finish_check += 1
                else:
                    reconstruct_counter = 0
                    break
        
            ### If all of the columns have at least 25 matches, checking through the file is stopped early:
            if finish_check == len(reconstruct_check_list):  
                print('exit')
                break
                    
    align_filename = file.strip('.fastq')
    align_filename = 'Alignments\\' + align_filename + '.csv'
                
    reconstruct_df.to_csv(align_filename) 
        

In [ ]:
'''
Now I want to:
    1)
        Count the number of nucleotides for each aligned matches
    
    2) 
        Determine the variant(s) produced by the individual
    
    3)
        Reconstruct the possible mRNA preproinsulin for each individual
        
'''

# Creating a reference of the 10 individuals:
individual_ref = sample_reference[['Characteristics[individual]', 'Characteristics[disease]']].drop_duplicates()

# Getting all the filenames saved in the previous cell:
align_list = os.listdir('Alignments\\')

if os.path.exists('Counts\\') != True: ## Creating a Counts folder
    os.mkdir('Counts\\')
    
if os.path.exists('Reconstruct\\') != True: ## Creating a Reconstruct folder
    os.mkdir('Reconstruct\\')

for index, row in individual_ref.iterrows():
    individual = row['Characteristics[individual]']
    print(individual)
    
    concat_df = pd.DataFrame()
    
    for alignment_file in align_list:
        if individual in alignment_file:
            temp_df = pd.read_csv('Alignments\\' + alignment_file, index_col= 0, dtype= 'str')  
                ## dtype= 'str' is needed because there are np.NaN and strings as values
            
            if len(concat_df) == 0:
                concat_df = temp_df
            
            else:
                concat_df = pd.concat([concat_df, temp_df], ignore_index= True)
                
    ## Creating a DataFrame to count the nucleotides for each position
    nucleotide_df = pd.DataFrame(columns= list(range(0, 644)), index= ['A', 'T', 'G', 'C']) 
        
    for column in concat_df.columns:
        A_count = len(concat_df[concat_df[column] == 'A'])  ### The len() function works around needing to change np.NaN
        T_count = len(concat_df[concat_df[column] == 'T'])
        G_count = len(concat_df[concat_df[column] == 'G'])
        C_count = len(concat_df[concat_df[column] == 'C'])
        
        nucleotide_df.loc['A', int(column)] = A_count  ### Conversion of "column" to int is needed because the column names  
        nucleotide_df.loc['T', int(column)] = T_count  ### in nucleotide_df are int, while when temp_df was loaded, the 
        nucleotide_df.loc['G', int(column)] = G_count  ### column names were converted to strings
        nucleotide_df.loc['C', int(column)] = C_count
        
    nucleotide_df.to_csv('Counts\\' + individual + '.csv')
    
    # Reconstructing the mRNA for all variants:
    variant_df = pd.DataFrame(columns= list(range(0, 644)))
    
    variant_1_list = list(range(0, 42)) ## Variant 1, Exon 1
    variant_1_list.extend(list(range(224, 644)))
    
    variant_2_list = list(range(0, 67)) ## Variant 2, Exon 1
    variant_2_list.extend(list(range(224, 644)))
    
    variant_4_list = list(range(0, 101)) ## Variant 4, Exon 1
    variant_4_list.extend(list(range(224, 644)))
    
    variant_3_list = list(range(0, 644)) ## Variant 3: Remember that this is the one used as a reference for alignments
    
    variant_list = [variant_1_list, variant_2_list, variant_3_list, variant_4_list]
    
    variant_reconstruct_df = pd.DataFrame(columns= list(range(0, 644)), 
                                          index= ['variant_1', 'variant_2', 'variant_3', 'variant_4'])
    
    variant_counter = 0
    for variant in variant_list:
        for nucleotide_position in variant:
                ## Returning the index (A, T, G, or C) of the max count in that column
                max_count = nucleotide_df[nucleotide_position].max()
                
                if max_count == 0:
                    freq_nucleotide = ''
                    variant_reconstruct_df.iloc[variant_counter, nucleotide_position] = freq_nucleotide
                else:
                    freq_nucleotide = nucleotide_df[nucleotide_df[nucleotide_position] == max_count].index
                    variant_reconstruct_df.iloc[variant_counter, nucleotide_position] = freq_nucleotide[0]
        
        variant_counter += 1
    
    variant_reconstruct_df.to_csv('Reconstruct\\' + individual + '.csv')
        

In [ ]:
# Finding all the differences in mRNA sequences
variant_3_list = list(str(var_3_ref_mRNA_cDNA.seq))
counter = 0
for x in variant_3_list:
    compare_df.loc['reference', str(counter)] = x
    counter += 1

if os.path.exists('Summary\\') != True: ## Creating a Summary folder
    os.mkdir('Summary\\')
    
compare_df.to_csv('Summary\\mRNA Comparison - Unfilled.csv')

# Finding which nucleotide positions are different
difference_df = pd.DataFrame()

for nucleotide in list(range(0,644)):
    temp_splice = compare_df[[str(nucleotide)]]
    if len(set(compare_df[str(nucleotide)])) != 1:
        if len(difference_df) == 0:
            difference_df = temp_splice
        else:
            difference_df = pd.concat([difference_df, temp_splice], axis= 1)
            
difference_df.to_csv('Summary\\mRNA Comparison - Difference.csv')

# Filling any empty nucleotides positions with the reference mRNA sequence
compare_df_filled = compare_df.copy()
for nucleotide in list(range(0,644)):
    check_list = list(compare_df[str(nucleotide)])
    while "" in check_list:
        empty_position = list(compare_df[str(nucleotide)]).index("")
        compare_df_filled.iloc[empty_position, nucleotide] = compare_df_filled.loc["reference", str(nucleotide)]
        check_list = list(compare_df_filled[str(nucleotide)])
        
compare_df_filled.to_csv('Summary\\mRNA Comparison - Filled.csv')

# Translating the mRNA sequences to protein sequences
compare_protein_df = pd.DataFrame()

variant_1_list = list(range(238, 571)) ## Variant 1 Protein Coding Region (same region for all other varients)
variant_1_list = [str(x) for x in variant_1_list]

for patient in list(compare_df_filled.index):
    temp_string = ""

    for nucleotide in list(compare_df_filled.loc[patient, variant_1_list]):
        temp_string += nucleotide
        
    protein_seq = str(Seq(temp_string).translate())
    
    for column in list(range(len(protein_seq))):
        compare_protein_df.loc[patient, column] = protein_seq[column]
        
compare_protein_df.to_csv('Summary\\Protein Comparison.csv')
        
# Comparing the protein sequences to find any differences
difference_protein_df = pd.DataFrame()
for aminoacid in list(range(110)):
    temp_splice = compare_protein_df[[(aminoacid)]]
    if len(set(compare_protein_df[(aminoacid)])) != 1:
        if len(difference_protein_df) == 0:
            difference_protein_df = temp_splice
        else:
            difference_protein_df = pd.concat([difference_protein_df, temp_splice], axis= 1)
            
difference_protein_df.to_csv('Summary\\Protein Comparison - Difference.csv')  ## There are no differences in protein sequences